# Intenssion:
ML-Models need numbers to work with. The webscraping does not provide enough data/features. We build some features by our own. 

1. built feature is the distance to the next train station
---
- **input**: address
- **output**: distance in meters


In [2]:
# Libraries
import os
import json
import urllib
import requests
import pandas as pd
from math import radians, cos, sin, sqrt, atan2



In [3]:
address = "8400 Winterthur, Theaterstrasse 17.1"
#address = input("Enter the address: ")

In [4]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": address,
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
print(f"{base_url}{urllib.parse.urlencode(parameters)}")

https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText=8400+Winterthur%2C+Theaterstrasse+17.1&origins=address&type=locations


In [72]:
parameters = {"searchText": address,
              "origins": "address",
              "type": "locations",
             }
response = requests.get(f"{base_url}", params=parameters)
response.json()

{'error': {'code': 400,
  'message': 'Please provide a type parameter. Possible values are locations, layers, featuresearch'},
 'success': False}

In [5]:
# Server request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data
# can return several results, so we need to select the first one. It is the one closer to the input address. try: "8400 Winterthur, Theaterstrasse 17.1" vs "8400 Winterthur, Theaterstrasse 17"

{'results': [{'attrs': {'detail': 'theaterstrasse 17.1 8400 winterthur 230 winterthur ch zh',
    'featureId': '201032451_0',
    'geom_quadindex': '030010211112102121200',
    'geom_st_box2d': 'BOX(697167.118926371 262226.7588384855,697167.118926371 262226.7588384855)',
    'label': 'Theaterstrasse 17.1 <b>8400 Winterthur</b>',
    'lat': 47.50360870361328,
    'lon': 8.728364944458008,
    'num': 171,
    'objectclass': '',
    'origin': 'address',
    'rank': 7,
    'x': 262226.75,
    'y': 697167.125,
    'zoomlevel': 10},
   'id': 1573119,
   'weight': 4},
  {'attrs': {'detail': 'theaterstrasse 17 8400 winterthur 230 winterthur ch zh',
    'featureId': '2323240_0',
    'geom_quadindex': '030010211112013212300',
    'geom_st_box2d': 'BOX(697128.5668860576 262216.11449080077,697128.5668860576 262216.11449080077)',
    'label': 'Theaterstrasse 17 <b>8400 Winterthur</b>',
    'lat': 47.503517150878906,
    'lon': 8.727851867675781,
    'num': 17,
    'objectclass': '',
    'origin': '

In [6]:
data['results'][0]

{'attrs': {'detail': 'theaterstrasse 17.1 8400 winterthur 230 winterthur ch zh',
  'featureId': '201032451_0',
  'geom_quadindex': '030010211112102121200',
  'geom_st_box2d': 'BOX(697167.118926371 262226.7588384855,697167.118926371 262226.7588384855)',
  'label': 'Theaterstrasse 17.1 <b>8400 Winterthur</b>',
  'lat': 47.50360870361328,
  'lon': 8.728364944458008,
  'num': 171,
  'objectclass': '',
  'origin': 'address',
  'rank': 7,
  'x': 262226.75,
  'y': 697167.125,
  'zoomlevel': 10},
 'id': 1573119,
 'weight': 4}

In [7]:
Latitude = data['results'][0]['attrs']['lat']
Latitude

47.50360870361328

In [8]:
Longitude = data['results'][0]['attrs']['lon']
Longitude

8.728364944458008

In [6]:
def get_coordinates(address):
    base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"
    parameters = {"searchText": address,
                "origins": "address",
                "type": "locations",
                }
    r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
    data = json.loads(r.content)
    if data['results'] == [] or len(data['results']) == 0:
        return None, None
    try:
        Latitude = data['results'][0]['attrs']['lat']
        Longitude = data['results'][0]['attrs']['lon']
    except:
        return None, None
    return Latitude, Longitude  # return a tuple

coordinates = get_coordinates(input("Enter the address: "))
coordinates

KeyError: 'fuzzy'

In [88]:
def get_distance_to_station(Latitude, Longitude):
    """
    This function determines the closesd trainstation based on the input coordinates and returns the distance between the given Point and the nearest station.
    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        'format': 'json',
        'lat': Latitude,
        'lon': Longitude,
        'zoom': 18,
        'addressdetails': 1
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'address' in data and 'town' in data['address']:
        Town = data['address']['town']
    elif 'address' in data and 'city' in data['address']:
        Town = data['address']['city']
    else:
        return "No town found"
    
    url = "http://transport.opendata.ch/v1/locations"
    params = {
        'type': 'station',
        'query': Town
    }
    data = requests.get(url, params=params).json()
    station_lat = data["stations"][0]["coordinate"]["x"]
    station_lon = data["stations"][0]["coordinate"]["y"]

    # Calculate the distance between the two points with pythagoras
    # Can not apply pythagoras directly, because the earth is not flat
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(radians, [Latitude, Longitude, station_lat, station_lon])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    radius_of_earth_km = 6371.0
    distance = radius_of_earth_km * c

    return distance

Since the earth is not flat, the Pythagorean theorem would not be accurate for calculating distances between geographical coordinates. The Haversine formula or other spherical geometry methods are more appropriate for that purpose.


![Image description](resources\Haversine.png)

[Image Source](https://www.google.com/imgres?imgurl=https%3A%2F%2Fwww.vcalc.com%2Fattachments%2Fe6d11679-da27-11e2-8e97-bc764e04d25f%2F220px-Law-of-haversines.svg.png&tbnid=IWPxBR6bJWrqlM&vet=12ahUKEwjcuMO1-JmFAxU9kP0HHZ3sBuoQMygQegQIARBu..i&imgrefurl=https%3A%2F%2Fwww.vcalc.com%2Fwiki%2Fvcalc%2Fhaversine-distance&docid=MOeiC0D7gb9SqM&w=220&h=220&q=Haversine%20formula&ved=2ahUKEwjcuMO1-JmFAxU9kP0HHZ3sBuoQMygQegQIARBu)

In [89]:
get_distance_to_station(coordinates[0], coordinates[1])

1.0172157309429812